
# swift
[docs](https://swift.readthedocs.io/zh-cn/latest/GetStarted/%E5%BF%AB%E9%80%9F%E5%BC%80%E5%A7%8B.html) ------ [github](https://github.com/modelscope/ms-swift)

## 安装
### Wheel包安装
可以使用pip进行安装：
```bash
    # 推荐
    pip install 'ms-swift'
    # 使用评测
    pip install 'ms-swift[eval]' -U
    # 全能力
    pip install 'ms-swift[all]' -U
```

### 源代码安装
```bash
    pip install git+https://github.com/modelscope/ms-swift.git

    # 全能力
    # pip install "git+https://github.com/modelscope/ms-swift.git#egg=ms-swift[all]"

    git clone https://github.com/modelscope/ms-swift.git
    cd ms-swift
    pip install -e .

    # 全能力
    # pip install -e '.[all]'
    旧版本
    SWIFT在3.0版本开始进行了不兼容式重构，如果需要使用2.x旧版本，请执行如下命令进行安装：

    pip install ms-swift==2.*
```

### 镜像
```bash
# swift3.5.3
modelscope-registry.cn-hangzhou.cr.aliyuncs.com/modelscope-repo/modelscope:ubuntu22.04-cuda12.4.0-py310-torch2.6.0-vllm0.8.5.post1-modelscope1.27.1-swift3.5.3
modelscope-registry.cn-beijing.cr.aliyuncs.com/modelscope-repo/modelscope:ubuntu22.04-cuda12.4.0-py310-torch2.6.0-vllm0.8.5.post1-modelscope1.27.1-swift3.5.3
modelscope-registry.us-west-1.cr.aliyuncs.com/modelscope-repo/modelscope:ubuntu22.04-cuda12.4.0-py310-torch2.6.0-vllm0.8.5.post1-modelscope1.27.1-swift3.5.3

# swift3.4.1.post1
modelscope-registry.cn-hangzhou.cr.aliyuncs.com/modelscope-repo/modelscope:ubuntu22.04-cuda12.4.0-py311-torch2.6.0-vllm0.8.5.post1-modelscope1.26.0-swift3.4.1.post1
modelscope-registry.cn-beijing.cr.aliyuncs.com/modelscope-repo/modelscope:ubuntu22.04-cuda12.4.0-py311-torch2.6.0-vllm0.8.5.post1-modelscope1.26.0-swift3.4.1.post1
modelscope-registry.us-west-1.cr.aliyuncs.com/modelscope-repo/modelscope:ubuntu22.04-cuda12.4.0-py311-torch2.6.0-vllm0.8.5.post1-modelscope1.26.0-swift3.4.1.post1

```
启动docker：
```bash
docker run -it -p 5053:5053 --gpus all --name swift-container --privileged -v /ssd3/guzhen/:/workspace --shm-size=64g --ulimit nofile=65525:65525 --ulimit nproc=65525:65525 modelscope-registry.cn-beijing.cr.aliyuncs.com/modelscope-repo/modelscope:ubuntu22.04-cuda12.4.0-py310-torch2.6.0-vllm0.8.5.post1-modelscope1.27.1-swift3.5.3 /bin/bash
```

### 启动脚本：
```bash
#!/bin/bash

# 指定4张GPU
export CUDA_VISIBLE_DEVICES=3,4,5,6

# 模型和数据路径
MERGED_MODEL_DIR="./ai_camera_question_lora_V4_amv-9pnvnurt2c45_2025-07-07-19:15:03/checkpoints_merged_lora"
DATASET_PATH="./dpo_train_data_1.jsonl"

# 检查路径是否存在
if [ ! -d "$MERGED_MODEL_DIR" ]; then
    echo "错误: 模型目录不存在 - $MERGED_MODEL_DIR"
    exit 1
fi

if [ ! -f "$DATASET_PATH" ]; then
    echo "错误: 数据集文件不存在 - $DATASET_PATH"
    exit 1
fi

# 创建DeepSpeed配置文件（如果不存在）
cat > ds_config.json << EOF
{
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu"
        },
        "offload_param": {
            "device": "cpu"
        },
        "stage3_gather_16bit_weights_on_model_save": true
    },
    "gradient_checkpointing": {
        "enabled": true
    },
    "bf16": {
        "enabled": true
    },
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": 2
}
EOF

# 分布式启动（启用ZeRO-3和梯度检查点）
torchrun --nproc_per_node=4 /usr/local/bin/swift rlhf \
    --rlhf_type dpo \
    --model "$MERGED_MODEL_DIR" \
    --train_type full \
    --dataset "$DATASET_PATH" \
    --torch_dtype bfloat16 \
    --num_train_epochs 2 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 2 \
    --freeze_vit true \
    --learning_rate 1e-5 \
    --gradient_accumulation_steps 2 \
    --eval_steps 30 \
    --save_steps 30 \
    --save_total_limit 10 \
    --logging_steps 5 \
    --max_length 2048 \
    --output_dir ./ \
    --warmup_ratio 0.05 \
    --save_only_model true \
    --dataloader_num_workers 4 \
    --dataset_num_proc 4 \
    --attn_impl flash_attn \
    --logging_dir ./runs \
    --deepspeed ds_config.json \
    --ignore_args_error True > ./runs/run.log 2>&1
```

# Unsloth
[部署微调](https://blog.csdn.net/weixin_58447899/article/details/146423437)


# LLaMA-Factory
[部署微调](https://zhuanlan.zhihu.com/p/26993872051)

[docs](https://qwen.readthedocs.io/zh-cn/latest/training/llama_factory.html)
